<a href="https://colab.research.google.com/github/markoo26/thehappymountain/blob/main/Experimentation_(part_3_Response_Surface_Methodology).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 🚚 Imports

import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [4]:
#@title 🌱 Random seed setup

np.random.seed(42)

In [2]:
#@title 💰 Markout profit definition and simulation for 10k runs

def markout_profit(threshold):
    cost = 1
    pps = 1
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = pps*np.abs(signal) - cost + eps
    else:
        profit = 0
    return profit

profit = np.array([markout_profit(threshold=1) for _ in range(10000)])

df = pd.DataFrame({
    "profit": profit,
})

df = df[df['profit'] != 0]

px.histogram(df)

In [3]:
#@title 🧮 Num ind experiments calculation
#@markdown Add plot that shows how num_ind changes based on params
sd_delta = 1.2 #@param
prac_sig = 0.03 #@param
num_ind = (2.48 * sd_delta/prac_sig)**2
num_ind = (3.08 * sd_delta/prac_sig)**2

print("sd_delta =", sd_delta, profit.std())
print("prac_sig = ", prac_sig, .2 * profit.mean())
print("num_ind =", num_ind)
print("num_ind =", num_ind)

sd_delta = 1.2 1.1998446977206074
prac_sig =  0.03 0.030883144321749535
num_ind = 9840.640000000001
num_ind = 15178.239999999998


In [25]:
#@title ⚒️ Functions

def run_experiment(num_ind, thresholds):
    individual_measurements = {
     threshold: [] for threshold in thresholds
    }
    done = set()
    while True:
        threshold = np.random.choice(thresholds)
        profit = markout_profit(threshold)
        individual_measurements[threshold].append(profit)
        if (len(individual_measurements[threshold])
            >= num_ind):
            done.add(threshold)
        if len(done)==len(thresholds):
            break

    aggregate_measurements = []
    standard_errors = []
    for threshold in thresholds:
        ims = np.array(individual_measurements[threshold])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )

    return aggregate_measurements, standard_errors

def linear_regression(thresholds, aggregate_measurements):
    x = thresholds
    y = aggregate_measurements
    X = np.array([np.ones(len(y)), x, x**2]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta


def interpolate(thresholds, beta):
    xhat = np.arange(thresholds.min(), thresholds.max()+1e-6, .01)
    XHat = np.array([np.ones(len(xhat)), xhat, xhat**2]).T
    yhat = XHat @ beta
    return xhat, yhat


def optimize(thresholds, beta):
    xhat, yhat = interpolate(thresholds, beta)
    i = np.where(yhat==yhat.max())[0][0]
    return xhat[i], yhat[i]

def markout_profit_2D(threshold, order_size):
    cost = 1
    pps = 1
    asc = 0.001*np.exp(2*order_size)
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = order_size*(pps*np.abs(signal) - cost + eps) - asc
    else:
        profit = 0
    return profit

def design_ccd(thresholds, order_sizes):
    parameters = [
        (threshold, order_size)
        for threshold in thresholds
        for order_size in order_sizes
    ]
    return parameters

def run_experiment_2D(num_ind, parameters):
    individual_measurements = {
      parameter: [] for parameter in parameters
    }
    done = set()
    while True:
        parameter = parameters[np.random.choice(len(parameters))]
        threshold, order_size = parameter
        profit = markout_profit_2D(threshold, order_size)
        individual_measurements[parameter].append(profit)
        if (len(individual_measurements[parameter])
            >= num_ind):
            done.add(parameter)
        if len(done) == len(individual_measurements):
            break

    aggregate_measurements = []
    standard_errors = []
    for parameter in parameters:
        ims = np.array(individual_measurements[parameter])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )

    return aggregate_measurements, standard_errors

def linear_regression_2D(parameters, aggregate_measurements):
    parameters = np.array(parameters)
    x0 = parameters[:,0]
    x1 = parameters[:,1]
    y = np.array(aggregate_measurements)
    X = np.array([np.ones(len(y)), x0, x1, x0**2, x1**2, x0*x1]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta

def interpolate_2D(parameters, beta):
    parameters = np.array(parameters)
    x0_values = np.arange(parameters[:,0].min(), parameters[:,0].max()+1e-6, .01)
    x1_values = np.arange(parameters[:,1].min(), parameters[:,1].max()+1e-6, .01)
    x0hat_2d, x1hat_2d = np.meshgrid(x0_values, x1_values)
    x0hat = x0hat_2d.flatten()
    x1hat = x1hat_2d.flatten()
    XHat = np.array([np.ones(len(x0hat)), x0hat, x1hat, x0hat**2, x1hat**2, x0hat*x1hat]).T
    yhat = XHat @ beta
    yhat_2d = np.reshape(yhat, (len(x1_values), len(x0_values)))
    return x0hat_2d, x1hat_2d, yhat_2d

def optimize_2D(parameters, beta):
    x0hat, x1hat, yhat = interpolate_2D(parameters, beta)
    i = np.where(yhat==yhat.max())
    return x0hat[i][0], x1hat[i][0], yhat[i][0]

In [7]:
#@title 🧃Run experiments with 3 different thresholds and do a boxplot
thresholds = np.array([0.5, 1.0, 1.5])
aggregate_measurements, standard_errors = run_experiment(15000, thresholds)


df = pd.DataFrame({
    "aggregate_measurements": aggregate_measurements,
    "standard_errors": standard_errors,
    "thresholds": thresholds,
})

df['error_plus'] = df['standard_errors'] / 2
df['error_minus'] = df['standard_errors'] / 2

# Renaming fig object to reuse it below
fig_error = px.scatter(
    df,
    x="thresholds",
    y="aggregate_measurements",
    error_y="error_plus",
    error_y_minus="error_minus"
)
fig_error.show()


In [10]:
#@title 🪢 Linear regression & interpolate

beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

xhat, yhat = interpolate(thresholds, beta)

df = pd.DataFrame({
    "xhat": xhat,
    "yhat": yhat
})

fig1 =px.line(df, x='xhat', y='yhat')
fig1.update_traces(line=dict(dash='dot'))

fig3 = go.Figure(data=fig1.data + fig_error.data)
fig3.show()

[-0.15795336  0.59403064 -0.26773906]


In [13]:
#@title 🎯 Optimize interpolate + find optimum

beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

threshold_opt, estimated_max_profit = optimize(thresholds, beta)
print(f"Optimal threshold: {threshold_opt}, estimated_max_profit: {estimated_max_profit}")

[-0.15795336  0.59403064 -0.26773906]
Optimal threshold: 1.1100000000000005, estimated_max_profit: 0.1715393526194155


In [14]:
#@title 🔫 Linear regression & interpolate with optimum

beta = linear_regression(thresholds, aggregate_measurements)

xhat, yhat = interpolate(thresholds, beta)

df = pd.DataFrame({
    "xhat": xhat,
    "yhat": yhat
})

fig1 =px.line(df, x='xhat', y='yhat')
fig1.update_traces(line=dict(dash='dot'))

# Add a trace for optimal values
df = pd.DataFrame({'threshold_opt': [threshold_opt], 'estimated_max_profit': [estimated_max_profit], 'size': [20] })
fig4 = px.scatter(df, x='threshold_opt', y='estimated_max_profit', size='size')
fig4.update_traces(marker=dict(color='red'))

fig3 = go.Figure(data=fig1.data + fig_error.data + fig4.data)
fig3.show()

[-0.15795336  0.59403064 -0.26773906]


In [16]:
#@title 💂‍♂️ Validate the parameter
aggregate_measurement, standard_error = run_experiment(15000, [threshold_opt])
print(f"Aggregate measurement confidence bounds (mean +/- 2 * std): {round(aggregate_measurement[0]-2*standard_error[0],4)}, {round(aggregate_measurement[0]+2*standard_error[0],4)}")

Aggregate measurement confidence bounds (mean +/- 2 * std): 0.1369, 0.1716


In [18]:
#@title 🥅 Plot cross join grid of thresholds and order_sizes
parameters = design_ccd(thresholds=[0.5, 1.0, 1.5], order_sizes=[1, 1.5, 2])
df = pd.DataFrame({
    "thresholds": [t[0] for t in parameters],
    "order_sizes": [t[1] for t in parameters]
})

fig0 = px.scatter(df, x='thresholds', y='order_sizes')
fig0.show()

In [21]:
#@title 🪓 Run markout_profit_2D - show mean, std and nonzero profits

profit = np.array([markout_profit_2D(threshold=1, order_size=1) for _ in range(10000)])
i = np.where(profit!=0)[0]
print("Total observations: 10000")
print(f"Non-zero profits: {len(i)}")
print(f"Percentage of zero profits {(len(profit)-len(i))/len(profit)}")
print(f"Profit mean: {round(profit.mean(),4)} and std: {round(profit.std(), 4)}")

Total observations: 10000
Non-zero profits: 3095
Percentage of zero profits 0.6905
Profit mean: 0.1718 and std: 1.2057


In [23]:
#@title 📉 Boxplot for 2D experiment

parameters = design_ccd(thresholds=[0.5, 1.0, 1.5], order_sizes=[1, 1.5, 2])
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)
legend = [f"tr: {t[0]}, os: {t[1]}" for t in parameters]
df = pd.DataFrame({
    "aggregate_measurements": aggregate_measurements,
    "standard_errors": standard_errors,
    "params": legend,
})

df['error_plus'] = df['standard_errors'] / 2
df['error_minus'] = df['standard_errors'] / 2

# Renaming fig object to reuse it below
fig_error = px.scatter(
    df,
    x="params",
    y="aggregate_measurements",
    error_y="error_plus",
    error_y_minus="error_minus"
)
fig_error.show()


In [105]:
#@title 🌎 Surface 2D plot
beta = linear_regression_2D(parameters, aggregate_measurements)
print(f"Fitted betas: {beta}")

x0hat, x1hat, yhat = interpolate_2D(parameters, beta)

fig = go.Figure(data=go.Contour(z=yhat))


fig5 = go.Figure(data=fig.data)


fig6 = go.Figure(data=fig5.data)
x_ticks = [round(0.5 + x*0.01,2) for x in range(100)]
y_ticks = [round(1 + x*0.01,2) for x in range(100)]
fig6.update_xaxes(
    tickvals=[x for x in range(100)],
    ticktext= x_ticks,
    nticks=5
)

fig6.update_yaxes(
    tickvals=[x for x in range(100)],
    ticktext= y_ticks,
    nticks=5
)
fig6.update_layout(height=500, width=1000)

fig6.show()


In [27]:
#@title 🏁 Show results for all grid-searched values

beta = linear_regression_2D(parameters, aggregate_measurements)
threshold_opt, order_size_opt, estimated_max_profit = optimize_2D(parameters, beta)
print (threshold_opt, order_size_opt, estimated_max_profit)


parameters = design_ccd(thresholds=[.5,  1,  1.5], order_sizes=[2.5, 3, 3.5])
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)
aggregate_measurements_prev, standard_errors_prev = aggregate_measurements, standard_errors

aggregate_measurements_prev

1.1700000000000006 2.869999999999992 0.44511972633354757


[np.float64(0.08425293161441551),
 np.float64(0.02880132507480998),
 np.float64(-0.4166640718904692),
 np.float64(0.353427831663405),
 np.float64(0.41050639238981573),
 np.float64(0.23058506091607894),
 np.float64(0.2761481804997162),
 np.float64(0.2846094298687624),
 np.float64(0.31220399904518714)]